## Some charts about COVID-19 
Provided daily via Telegram channel: https://t.me/covid19charts. You can subscribe and receive automatic updates.

Some basics charts about COVID-19 from https://github.com/CSSEGISandData/COVID-19.git
**Cumulative, daily and percentage** charts on **Region/Country** with more than **500 Confirmed**. 

Code available at https://github.com/andreaaizza/covid-19-basic-charts

In [ ]:
import pandas as pd
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [ ]:
def load_file(f):
    # load
    df = pd.read_csv(f)
    df=df.drop(columns=['Lat','Long'])
    
    # 'Location' index (sum()) as joint 'Country/Region','Province/State'
    df.loc[df['Province/State'].isna(),'Location']=df.loc[
        df['Province/State'].isna(),'Country/Region']
    df.loc[~df['Province/State'].isna(),'Location']=df.loc[
        ~df['Province/State'].isna(),'Country/Region'] + ' - ' + df.loc[
        ~df['Province/State'].isna(),'Province/State']
    return df.groupby('Location').sum()

# select significant_ones
df=load_file("./COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv")
# get only >500
df=df.sort_values(df.columns[len(df.columns)-1], ascending=False)
last_col=df.columns[len(df.columns)-1]
significant_ones=df[df[last_col]>1000].index

# load all files
df_suff=['confirmed','deaths','recovered']
df_suff_without_confirmed=['Deaths','Recovered']
df=pd.DataFrame()
for s in df_suff:
    d=load_file("./COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_"+s+"_global.csv")
    d['Type']=s
    df=df.append(d)
# set multiindex
df=df.set_index('Type', append=True)

### Cumulative Confirmed only

In [ ]:
fig = go.Figure()
for loc in significant_ones:
    fig.add_trace(
        go.Scatter(
            x=df.columns, 
            y=df[
                (df.index.get_level_values('Type')=='confirmed') & 
                (df.index.get_level_values('Location')==loc)
                ].iloc[0],
            name=loc,
            mode='lines+markers'
        )
    )
fig.update_layout(xaxis_rangeslider_visible=True)
fig.show()

In [ ]:
def plotCounties(f): 
    plt_size=len(significant_ones)
    for i in range(0,plt_size): 
        country=significant_ones[i]
        f(df, i, country)

### Per country

In [ ]:
def p_cum(df, i, loc):
    try:       
        fig = go.Figure()
        for t in df_suff:
            d=df[
                (df.index.get_level_values('Type')==t) & 
                (df.index.get_level_values('Location')==loc)
                ]
            if len(d)>0:
                fig.add_trace(
                    go.Scatter(
                        x=df.columns, 
                        y=d.iloc[0],
                        name=t,
                        mode='lines+markers'
                    )
                )
        fig.update_layout(title_text=loc+' - Cumulative')
        fig.show()

        fig = go.Figure()
        for t in df_suff:
            d=df[
                (df.index.get_level_values('Type')==t) & 
                (df.index.get_level_values('Location')==loc)
                ]
            if len(d)>0:
                fig.add_trace(
                    go.Scatter(
                        x=df.columns, 
                        y=d.iloc[0]-d.iloc[0].shift(1),
                        name=t,
                        mode='lines+markers'
                    )
                )
        fig.update_layout(title_text=loc+' - Daily')
        fig.show()
        
    except:
        print("Cannot chart")
    
plotCounties(p_cum)